# Сегментация кубов и параллелепипедов с определением связей

Этот ноутбук обучает модель Mask R-CNN для:
1. **Instance Segmentation** красных кубов (3×3×3 см) и зелёных параллелепипедов (0.3×0.3×2 см)
2. **Определение связей** какой параллелепипед принадлежит какому кубу через `parent_id`

Датасет в формате COCO с дополнительными полями:
- `instance_id` — уникальный ID объекта в сцене
- `parent_id` — для параллелепипедов указывает на `instance_id` родительского куба
- `visibility_ratio` — % видимости объекта (учитывает окклюзии)

## 1. Установка зависимостей

In [ ]:
!pip install torch torchvision
!pip install pycocotools
!pip install opencv-python-headless
!pip install albumentations
!pip install matplotlib
!pip install scipy


[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import gc
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
import os
import json
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from collections import defaultdict

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision.models.detection import maskrcnn_resnet50_fpn, MaskRCNN_ResNet50_FPN_Weights
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
import torchvision.transforms as T

from scipy.optimize import linear_sum_assignment

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. Конфигурация

In [ ]:
# --- ЛОГИКА ЗАГРУЗКИ ДАТАСЕТА ---
import os
import subprocess
import sys

REPO_URL = "https://github.com/SergKurchev/strawberry_peduncle_segmentation.git"
REPO_NAME = "strawberry_peduncle_segmentation"
KAGGLE_PATH = "/kaggle/input/strawberry-peduncle-segmentation"

# 1. Проверяем наличие на Kaggle (если запуск там)
if os.path.exists(KAGGLE_PATH):
    DATASET_PATH = KAGGLE_PATH
    print(f"✅ Используется датасет из Kaggle Input: {DATASET_PATH}")
else:
    # 2. Если на Kaggle нет, клонируем репозиторий из GitHub
    if not os.path.exists(REPO_NAME):
        print(f"🚀 Клонируем репозиторий из GitHub: {REPO_URL}...")
        subprocess.run(["git", "clone", REPO_URL])
    else:
        print(f"✅ Репозиторий {REPO_NAME} уже существует.")
    
    # Определение пути к датасету внутри репо
    # Проверяем два варианта структуры (с вложенной папкой и без)
    opt1 = os.path.join(REPO_NAME, "strawberry_peduncle_segmentation", "dataset")
    opt2 = os.path.join(REPO_NAME, "dataset")
    
    if os.path.exists(opt1):
        DATASET_PATH = opt1
    elif os.path.exists(opt2):
        DATASET_PATH = opt2
    else:
        DATASET_PATH = REPO_NAME
        print(f"⚠️ Внимание: Папка 'dataset' не найдена, используем корень репозитория.")

print(f"📍 Итоговый путь к датасету: {DATASET_PATH}")

IMAGES_PATH = os.path.join(DATASET_PATH, "images")
MASKS_PATH = os.path.join(DATASET_PATH, "masks")
ANNOTATIONS_PATH = os.path.join(DATASET_PATH, "annotations.json")

# Проверка наличия изображений
if not os.path.exists(IMAGES_PATH):
    print(f"❌ ВНИМАНИЕ: Папка с изображениями не найдена по пути: {IMAGES_PATH}")
    print(f"   Если вы используете GitHub, убедитесь, что изображения добавлены в репозиторий.")
    print(f"   Если изображения на Kaggle, добавьте датасет в Input.")

# Гиперпараметры
NUM_CLASSES = 3  # background + red_cube + green_parallelepiped
BATCH_SIZE = 4
NUM_EPOCHS = 30
LEARNING_RATE = 0.005
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Категории
CATEGORIES = {
    0: "background",
    1: "red_cube",
    2: "green_parallelepiped"
}

# Размер обучающей выборки
TRAIN_RATIO = 0.8


## 3. Анализ датасета

In [ ]:
# Загрузка и анализ аннотаций
with open(ANNOTATIONS_PATH, 'r') as f:
    data = json.load(f)

print(f"📊 Статистика датасета:")
print(f"   Изображений: {len(data['images'])}")
print(f"   Аннотаций: {len(data['annotations'])}")
print(f"   Категорий: {len(data['categories'])}")

# Подсчёт по категориям
category_counts = defaultdict(int)
for ann in data['annotations']:
    category_counts[ann['category_id']] += 1

print(f"\n📦 Объектов по категориям:")
for cat in data['categories']:
    print(f"   {cat['name']}: {category_counts[cat['id']]}")

# Проверка связей parent_id
cubes = [a for a in data['annotations'] if a['category_id'] == 1]
paras = [a for a in data['annotations'] if a['category_id'] == 2]

# Создаём маппинг instance_id куба
cube_instance_ids = set()
for ann in data['annotations']:
    if ann['category_id'] == 1:
        cube_instance_ids.add((ann['image_id'], ann['instance_id']))

# Проверяем parent_id параллелепипедов
valid_parents = 0
for ann in data['annotations']:
    if ann['category_id'] == 2 and ann['parent_id'] > 0:
        if (ann['image_id'], ann['parent_id']) in cube_instance_ids:
            valid_parents += 1

print(f"\n🔗 Связи parent_id:")
print(f"   Параллелепипедов с валидным parent_id: {valid_parents}/{len(paras)}")

# Проверка visibility_ratio если есть
if 'visibility_ratio' in data['annotations'][0]:
    vis_ratios = [a.get('visibility_ratio', 1.0) for a in data['annotations']]
    print(f"\n👁️ Visibility ratio:")
    print(f"   Min: {min(vis_ratios):.2f}, Max: {max(vis_ratios):.2f}, Avg: {np.mean(vis_ratios):.2f}")

In [ ]:
# Визуализация примера из датасета (V4: ID Collision Fix)
import cv2
import numpy as np

sample_image_id = data['images'][0]['id']
sample_image_info = data['images'][0]

# Загружаем изображение и маску
img = Image.open(os.path.join(IMAGES_PATH, sample_image_info['file_name']))
mask = Image.open(os.path.join(MASKS_PATH, sample_image_info['file_name']))

# Аннотации для этого изображения
sample_anns = [a for a in data['annotations'] if a['image_id'] == sample_image_id]

fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# RGB изображение
axes[0].imshow(img)
axes[0].set_title('RGB Image')

# Маска (усиленная)
mask_np = np.array(mask)
enhanced_mask = np.clip(mask_np.astype(np.float32) * 30, 0, 255).astype(np.uint8)
axes[1].imshow(enhanced_mask)
axes[1].set_title('Segmentation Mask (enhanced)')

# RGB с bbox (Calculate from Mask!)
axes[2].imshow(img)

mask_img_np = np.array(mask.convert("RGB"))
print(f"\n📝 Objects found in mask:")

# Shared Logic Helper
def get_cleaned_obj(mask_img_np, seg_color):
    obj_mask = np.all(mask_img_np == seg_color, axis=2).astype(np.uint8)
    if obj_mask.sum() == 0: return None
    num_labels, labels_im, stats, centroids = cv2.connectedComponentsWithStats(obj_mask, connectivity=8)
    if num_labels <= 1: return None
    largest_label = 1
    max_area = stats[1, cv2.CC_STAT_AREA]
    for i in range(2, num_labels):
        if stats[i, cv2.CC_STAT_AREA] > max_area:
            max_area = stats[i, cv2.CC_STAT_AREA]
            largest_label = i
    if max_area < 50: return None # 50px Threshold
    
    # Get BBox & Center
    obj_mask = (labels_im == largest_label).astype(np.uint8)
    pos = np.where(obj_mask)
    xmin, xmax = np.min(pos[1]), np.max(pos[1])
    ymin, ymax = np.min(pos[0]), np.max(pos[0])
    bbox = (xmin, ymin, xmax-xmin, ymax-ymin)
    center = centroids[largest_label]
    return bbox, center

cube_objects = {}
all_objects = {}

for ann in sample_anns:
    res = get_cleaned_obj(mask_img_np, ann['segmentation_color'])
    if res:
        all_objects[ann['instance_id']] = (res, ann['category_id'])
        if ann['category_id'] == 1:
            cube_objects[ann['instance_id']] = res

for ann in sample_anns:
    if ann['instance_id'] not in all_objects: 
        continue
    
    (bbox, center), cat_id = all_objects[ann['instance_id']]
    (x, y, w, h) = bbox
    
    color = 'red' if cat_id == 1 else 'green'
    rect = plt.Rectangle((x, y), w, h, fill=False, edgecolor=color, linewidth=2)
    axes[2].add_patch(rect)

    # Draw associations (Targeting Cubes Only)
    if cat_id == 2 and ann['parent_id'] > 0:
        # Look in CUBE map
        if ann['parent_id'] in cube_objects:
             px, py = cube_objects[ann['parent_id']][1]
             cx, cy = center
             
             # CHECK DISTANCE
             dist = ((px-cx)**2 + (py-cy)**2)**0.5
             if dist < 200:
                 axes[2].plot([cx, px], [cy, py], 'y-', linewidth=2)

axes[2].set_title('Bounding Boxes with Associations (SANITIZED)')
plt.tight_layout()
plt.show()

print(f"\n📝 Аннотаций на изображении: {len(sample_anns)}")
for ann in sample_anns:
    cat_name = CATEGORIES[ann['category_id']]
    parent_info = f" -> parent: {ann['parent_id']}" if ann['parent_id'] > 0 else ""
    print(f"   [{ann['instance_id']}] {cat_name}{parent_info}")


## 4. Dataset класс

In [ ]:
class CubeParallelepipedDataset(Dataset):
    """Dataset для кубов и параллелепипедов с информацией о связях."""
    
    def __init__(self, images_path, masks_path, annotations_path, transforms=None):
        self.images_path = images_path
        self.masks_path = masks_path
        self.transforms = transforms
        
        # Загрузка аннотаций
        with open(annotations_path, 'r') as f:
            data = json.load(f)
        
        self.images_info = {img['id']: img for img in data['images']}
        
        # Группировка аннотаций по изображениям
        self.annotations_by_image = defaultdict(list)
        for ann in data['annotations']:
            self.annotations_by_image[ann['image_id']].append(ann)
        
        self.image_ids = list(self.images_info.keys())
    
    def __len__(self):
        return len(self.image_ids)
    
    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        image_info = self.images_info[image_id]
        
        # Загрузка изображения
        img_path = os.path.join(self.images_path, image_info['file_name'])
        image = Image.open(img_path).convert("RGB")
        image = np.array(image)
        
        # Загрузка маски
        mask_path = os.path.join(self.masks_path, image_info['file_name'])
        mask_image = np.array(Image.open(mask_path).convert("RGB"))
        
        # Получение аннотаций для этого изображения
        annotations = self.annotations_by_image[image_id]
        
        boxes = []
        labels = []
        masks = []
        instance_ids = []
        parent_ids = []
        
        # Maps for sanitation: Only Cubes (Cat 1) can be parents
        cube_centroids = {}
        processed_objects = []
        
        # Helper function 
        def process_mask(ann, mask_img):
            seg_color = ann['segmentation_color']
            obj_mask = np.all(mask_img == seg_color, axis=2).astype(np.uint8)
            if obj_mask.sum() == 0: return None, None, None
            
            import cv2
            num_labels, labels_im, stats, centroids = cv2.connectedComponentsWithStats(obj_mask, connectivity=8)
            if num_labels <= 1: return None, None, None
            
            largest_label = 1
            max_area = stats[1, cv2.CC_STAT_AREA]
            for i in range(2, num_labels):
                if stats[i, cv2.CC_STAT_AREA] > max_area:
                    max_area = stats[i, cv2.CC_STAT_AREA]
                    largest_label = i
            
            # Filter small noise (increased threshold)
            if max_area < 50: return None, None, None
            
            cleaned_mask = (labels_im == largest_label).astype(np.uint8)
            
            pos = np.where(cleaned_mask)
            xmin, xmax = np.min(pos[1]), np.max(pos[1])
            ymin, ymax = np.min(pos[0]), np.max(pos[0])
            if xmax <= xmin: xmax = xmin + 1
            if ymax <= ymin: ymax = ymin + 1
            
            center = centroids[largest_label]
            
            return cleaned_mask, [xmin, ymin, xmax, ymax], center

        # Pass 1: Extract all objects
        for ann in annotations:
             msk, box, center = process_mask(ann, mask_image)
             if msk is not None:
                 processed_objects.append((ann, msk, box, center))
                 # Only register CUBES as valid parents
                 if ann['category_id'] == 1:
                     cube_centroids[ann['instance_id']] = center
        
        # Pass 2: Commit targets with Sanitation
        for (ann, msk, box, center) in processed_objects:
            pid = ann['parent_id']
            
            if pid > 0:
                # 1. Check if Parent ID exists in CUBES map
                if pid not in cube_centroids:
                    pid = 0
                else:
                    # 2. Check Distance
                    pcx, pcy = cube_centroids[pid]
                    ccx, ccy = center
                    dist = ((pcx-ccx)**2 + (pcy-ccy)**2)**0.5
                    if dist > 200:
                        pid = 0 # Too far
            
            boxes.append(box)
            labels.append(ann['category_id'])
            instance_ids.append(ann['instance_id'])
            parent_ids.append(pid)
            masks.append(msk)
        
        # Проверка на пустые аннотации
        num_objs = len(boxes)
        
        if num_objs == 0:
            # Возвращаем пустые тензоры для изображений без объектов
            boxes = torch.zeros((0, 4), dtype=torch.float32)
            labels = torch.zeros((0,), dtype=torch.int64)
            masks = torch.zeros((0, image.shape[0], image.shape[1]), dtype=torch.uint8)
            instance_ids = torch.zeros((0,), dtype=torch.int64)
            parent_ids = torch.zeros((0,), dtype=torch.int64)
        else:
            boxes = torch.as_tensor(boxes, dtype=torch.float32)
            labels = torch.as_tensor(labels, dtype=torch.int64)
            masks = torch.as_tensor(np.array(masks), dtype=torch.uint8)
            instance_ids = torch.as_tensor(instance_ids, dtype=torch.int64)
            parent_ids = torch.as_tensor(parent_ids, dtype=torch.int64)
        
        image_id_tensor = torch.tensor([image_id])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0]) if num_objs > 0 else torch.zeros((0,))
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)
        
        target = {
            "boxes": boxes,
            "labels": labels,
            "masks": masks,
            "image_id": image_id_tensor,
            "area": area,
            "iscrowd": iscrowd,
            "instance_ids": instance_ids,
            "parent_ids": parent_ids
        }
        
        # Конвертация изображения в тензор
        image = torch.as_tensor(image, dtype=torch.float32).permute(2, 0, 1) / 255.0
        
        if self.transforms:
            image, target = self.transforms(image, target)
        
        return image, target


## 5. Модель Mask R-CNN

In [ ]:
def get_model(num_classes, pretrained=True):
    """Создаёт модель Mask R-CNN."""
    
    # Загружаем предобученную модель с новым API
    if pretrained:
        weights = MaskRCNN_ResNet50_FPN_Weights.DEFAULT
        model = maskrcnn_resnet50_fpn(weights=weights)
    else:
        model = maskrcnn_resnet50_fpn(weights=None)
    
    # Заменяем голову классификатора
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    
    # Заменяем голову масок
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    model.roi_heads.mask_predictor = MaskRCNNPredictor(
        in_features_mask, hidden_layer, num_classes
    )
    
    return model

## 6. Функции обучения

In [ ]:
import torch.cuda.amp as amp
import gc
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

def collate_fn(batch):
    """Кастомная функция для DataLoader."""
    return tuple(zip(*batch))

def train_one_epoch(model, optimizer, data_loader, device, epoch, scaler=None):
    """Обучение одной эпохи с AMP."""
    model.train()
    total_loss = 0
    
    for batch_idx, (images, targets) in enumerate(data_loader):
        images = list(img.to(device) for img in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        # AMP Context
        with torch.cuda.amp.autocast(enabled=(scaler is not None)):
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())
        
        optimizer.zero_grad()
        
        if scaler is not None:
            scaler.scale(losses).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            losses.backward()
            optimizer.step()
        
        total_loss += losses.item()
        
        if batch_idx % 10 == 0:
            print(f"Epoch [{epoch}] Batch [{batch_idx}/{len(data_loader)}] "
                  f"Loss: {losses.item():.4f}")
    
    return total_loss / len(data_loader)

@torch.no_grad()
def evaluate(model, data_loader, device):
    """Оценка модели (Optimized)."""
    model.eval()
    all_predictions = []
    all_targets = []
    
    for batch_idx, (images, targets) in enumerate(data_loader):
        images = list(img.to(device) for img in images)
        
        # AMP autocast for inference too to save memory
        with torch.cuda.amp.autocast():
            outputs = model(images)
        
        # Process outputs to save memory
        cleaned_outputs = []
        for output in outputs:
            # Remove masks to save huge amount of RAM - we only need boxes for association metric
            # Also keep only top 100
            if len(output['scores']) > 100:
                keep = torch.argsort(output['scores'], descending=True)[:100]
                box = output['boxes'][keep].cpu()
                lbl = output['labels'][keep].cpu()
                scr = output['scores'][keep].cpu()
            else:
                box = output['boxes'].cpu()
                lbl = output['labels'].cpu()
                scr = output['scores'].cpu()
            
            # Note: We DROP masks here intentionally for global stats
            cleaned_outputs.append({'boxes': box, 'labels': lbl, 'scores': scr})
            
        all_predictions.extend(cleaned_outputs)
        all_targets.extend(targets)
        
        if batch_idx % 10 == 0:
            gc.collect()
            if torch.cuda.is_available():
                 torch.cuda.empty_cache()
    
    return all_predictions, all_targets


## 7. Предсказание связей

In [ ]:
def predict_associations(boxes, labels, scores=None, score_threshold=0.5):
    """
    Предсказывает связи между параллелепипедами и кубами.
    
    Логика: параллелепипед принадлежит ближайшему кубу, который находится ПОД ним.
    
    Args:
        boxes: (N, 4) bbox в формате [x1, y1, x2, y2]
        labels: (N,) метки классов
        scores: (N,) опционально, confidence scores
        score_threshold: порог для фильтрации
    
    Returns:
        dict: {para_idx: cube_idx} - связи
    """
    if isinstance(boxes, torch.Tensor):
        boxes = boxes.cpu().numpy()
    if isinstance(labels, torch.Tensor):
        labels = labels.cpu().numpy()
    if scores is not None and isinstance(scores, torch.Tensor):
        scores = scores.cpu().numpy()
    
    # Находим индексы кубов и параллелепипедов
    cube_indices = np.where(labels == 1)[0]
    para_indices = np.where(labels == 2)[0]
    
    # Фильтруем по score если есть
    if scores is not None:
        cube_indices = [i for i in cube_indices if scores[i] >= score_threshold]
        para_indices = [i for i in para_indices if scores[i] >= score_threshold]
    
    associations = {}
    
    for para_idx in para_indices:
        para_box = boxes[para_idx]
        para_center_x = (para_box[0] + para_box[2]) / 2
        para_bottom = para_box[3]  # нижняя граница параллелепипеда
        
        min_dist = float('inf')
        best_cube_idx = None
        
        for cube_idx in cube_indices:
            cube_box = boxes[cube_idx]
            cube_center_x = (cube_box[0] + cube_box[2]) / 2
            cube_top = cube_box[1]  # верхняя граница куба
            
            # Параллелепипед должен быть НАД кубом (его низ близко к верху куба)
            # или пересекаться по вертикали
            vertical_dist = abs(para_bottom - cube_top)
            horizontal_dist = abs(para_center_x - cube_center_x)
            
            # Общее расстояние
            dist = np.sqrt(vertical_dist**2 + horizontal_dist**2)
            
            # Бонус если параллелепипед горизонтально по центру куба
            if cube_box[0] <= para_center_x <= cube_box[2]:
                dist *= 0.5  # уменьшаем расстояние
            
            if dist < min_dist:
                min_dist = dist
                best_cube_idx = cube_idx
        
        associations[int(para_idx)] = int(best_cube_idx) if best_cube_idx is not None else None
    
    return associations

## 8. Метрики для оценки связей

In [ ]:
def compute_iou(box1, box2):
    """Вычисляет IoU между двумя bbox."""
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    
    inter_area = max(0, x2 - x1) * max(0, y2 - y1)
    
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    
    union_area = box1_area + box2_area - inter_area
    
    return inter_area / union_area if union_area > 0 else 0


def compute_association_accuracy(predictions, targets, iou_threshold=0.5):
    """
    Вычисляет точность предсказания связей.
    
    Для каждого правильно обнаруженного параллелепипеда проверяет,
    правильно ли предсказан связанный куб.
    """
    correct_associations = 0
    total_associations = 0
    
    for pred, target in zip(predictions, targets):
        pred_labels = pred['labels'].cpu().numpy()
        pred_boxes = pred['boxes'].cpu().numpy()
        pred_scores = pred['scores'].cpu().numpy()
        
        target_labels = target['labels'].cpu().numpy()
        target_boxes = target['boxes'].cpu().numpy()
        target_instance_ids = target['instance_ids'].cpu().numpy()
        target_parent_ids = target['parent_ids'].cpu().numpy()
        
        # Предсказанные связи
        pred_associations = predict_associations(pred_boxes, pred_labels, pred_scores)
        
        # Для каждого GT параллелепипеда находим соответствующий предсказанный
        target_para_indices = np.where(target_labels == 2)[0]
        pred_para_indices = np.where(pred_labels == 2)[0]
        
        for target_idx in target_para_indices:
            target_box = target_boxes[target_idx]
            gt_parent_id = target_parent_ids[target_idx]
            
            # Находим лучший матч с предсказаниями
            best_iou = 0
            best_pred_idx = -1
            
            for pred_idx in pred_para_indices:
                if pred_scores[pred_idx] < 0.5:
                    continue
                pred_box = pred_boxes[pred_idx]
                iou = compute_iou(target_box, pred_box)
                if iou > best_iou:
                    best_iou = iou
                    best_pred_idx = pred_idx
            
            if best_iou >= iou_threshold and best_pred_idx >= 0:
                total_associations += 1
                
                # Проверяем правильность связи
                pred_cube_idx = pred_associations.get(best_pred_idx)
                
                if pred_cube_idx is not None:
                    pred_cube_box = pred_boxes[pred_cube_idx]
                    
                    # Находим GT куб с этим parent_id
                    gt_cube_indices = np.where(
                        (target_labels == 1) & (target_instance_ids == gt_parent_id)
                    )[0]
                    
                    if len(gt_cube_indices) > 0:
                        gt_cube_box = target_boxes[gt_cube_indices[0]]
                        cube_iou = compute_iou(pred_cube_box, gt_cube_box)
                        
                        if cube_iou >= iou_threshold:
                            correct_associations += 1
    
    accuracy = correct_associations / total_associations if total_associations > 0 else 0
    return accuracy, correct_associations, total_associations

## 9. Обучение модели

In [ ]:
# Создание датасетов
dataset = CubeParallelepipedDataset(
    images_path=IMAGES_PATH,
    masks_path=MASKS_PATH,
    annotations_path=ANNOTATIONS_PATH
)

# Разделение на train/val
train_size = int(TRAIN_RATIO * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(
    dataset, [train_size, val_size],
    generator=torch.Generator().manual_seed(42)
)

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")

In [ ]:
# Создание DataLoader
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=2
)

val_loader = DataLoader(
    val_dataset,
    batch_size=1,
    shuffle=False,
    collate_fn=collate_fn,
    num_workers=0
)

In [ ]:
# Создание модели
model = get_model(NUM_CLASSES, pretrained=True)
model.to(DEVICE)

# Оптимизатор
params = [p for p in model.parameters() if p.requires_grad]
optimizer = optim.SGD(params, lr=LEARNING_RATE, momentum=0.9, weight_decay=0.0005)

# Learning rate scheduler
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

print(f"Model loaded on {DEVICE}")

# Initialize scaler for AMP
scaler = torch.cuda.amp.GradScaler()


In [ ]:
# Цикл обучения
train_losses = []
val_accuracies = []
best_loss = float('inf')

for epoch in range(1, NUM_EPOCHS + 1):
    print(f"\n{'='*50}")
    print(f"Epoch {epoch}/{NUM_EPOCHS}")
    print(f"{'='*50}")
    
    # Обучение
    avg_loss = train_one_epoch(model, optimizer, train_loader, DEVICE, epoch, scaler)
    train_losses.append(avg_loss)
    
    print(f"\nAverage training loss: {avg_loss:.4f}")
    
    # Обновление learning rate
    lr_scheduler.step()
    
    # Сохранение лучшей модели
    if avg_loss < best_loss:
        best_loss = avg_loss
        torch.save(model.state_dict(), 'best_model.pth')
        print(f"Saved best model with loss: {best_loss:.4f}")
    
    # Периодическая оценка
    # Clear memory before evaluation
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    gc.collect()
    if torch.cuda.is_available(): torch.cuda.empty_cache()
    if epoch % 5 == 0:
        predictions, targets = evaluate(model, val_loader, DEVICE)
        acc, correct, total = compute_association_accuracy(predictions, targets)
        val_accuracies.append((epoch, acc))
        print(f"\nAssociation accuracy: {acc:.4f} ({correct}/{total})")

In [ ]:
# График обучения
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss
axes[0].plot(train_losses, 'b-', label='Training Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training Progress')
axes[0].legend()
axes[0].grid(True)

# Association accuracy
if val_accuracies:
    epochs, accs = zip(*val_accuracies)
    axes[1].plot(epochs, accs, 'g-o', label='Association Accuracy')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Accuracy')
    axes[1].set_title('Association Accuracy')
    axes[1].legend()
    axes[1].grid(True)

plt.tight_layout()
plt.savefig('training_progress.png')
plt.show()

## 10. Визуализация результатов

In [ ]:
def visualize_predictions(image, prediction, score_threshold=0.5):
    """Визуализирует предсказания модели с связями."""
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 8))
    
    # Оригинальное изображение с bbox
    img_np = image.cpu().numpy().transpose(1, 2, 0)
    axes[0].imshow(img_np)
    axes[0].set_title('Detections')
    
    labels = prediction['labels'].cpu().numpy()
    boxes = prediction['boxes'].cpu().numpy()
    scores = prediction['scores'].cpu().numpy()
    masks = prediction['masks'].cpu().numpy()
    
    colors = {1: 'red', 2: 'green'}
    
    # Рисуем bbox
    for i in range(len(labels)):
        if scores[i] >= score_threshold:
            box = boxes[i]
            label = labels[i]
            color = colors.get(label, 'blue')
            
            rect = plt.Rectangle(
                (box[0], box[1]), box[2] - box[0], box[3] - box[1],
                fill=False, edgecolor=color, linewidth=2
            )
            axes[0].add_patch(rect)
            axes[0].text(box[0], box[1] - 5, 
                        f"{CATEGORIES[label]}: {scores[i]:.2f}",
                        color=color, fontsize=8)
    
    # Маски с связями
    combined_mask = np.zeros((*img_np.shape[:2], 3))
    
    # Предсказываем связи
    associations = predict_associations(boxes, labels, scores, score_threshold)
    
    for i in range(len(labels)):
        if scores[i] >= score_threshold:
            mask = masks[i, 0] > 0.5
            label = labels[i]
            
            if label == 1:  # Куб - красный
                combined_mask[mask] = [1, 0, 0]
            elif label == 2:  # Параллелепипед - зелёный
                combined_mask[mask] = [0, 1, 0]
    
    axes[1].imshow(img_np)
    axes[1].imshow(combined_mask, alpha=0.5)
    axes[1].set_title('Masks with Associations')
    
    # Рисуем линии связей
    for para_idx, cube_idx in associations.items():
        if cube_idx is not None:
            if scores[para_idx] >= score_threshold and scores[cube_idx] >= score_threshold:
                para_box = boxes[para_idx]
                cube_box = boxes[cube_idx]
                
                para_center = [(para_box[0] + para_box[2]) / 2, (para_box[1] + para_box[3]) / 2]
                cube_center = [(cube_box[0] + cube_box[2]) / 2, (cube_box[1] + cube_box[3]) / 2]
                
                axes[1].plot([para_center[0], cube_center[0]], 
                            [para_center[1], cube_center[1]], 
                            'y-', linewidth=2)
                axes[1].plot(*para_center, 'yo', markersize=8)
                axes[1].plot(*cube_center, 'yo', markersize=8)
    
    plt.tight_layout()
    plt.show()
    
    return associations

In [ ]:
# Загрузка лучшей модели и визуализация
model.load_state_dict(torch.load('best_model.pth'))
model.eval()

# Получаем несколько примеров из validation set
sample_images = []
sample_targets = []

for i in range(min(5, len(val_dataset))):
    img, target = val_dataset[i]
    sample_images.append(img)
    sample_targets.append(target)

# Предсказания
with torch.no_grad():
    images_tensor = [img.to(DEVICE) for img in sample_images]
    predictions = model(images_tensor)

# Визуализация каждого примера
for i, (img, pred, target) in enumerate(zip(sample_images, predictions, sample_targets)):
    print(f"\n--- Sample {i + 1} ---")
    associations = visualize_predictions(img, pred)
    print(f"Predicted associations: {associations}")
    
    # Сравнение с GT
    gt_parent_ids = target['parent_ids'].numpy()
    gt_labels = target['labels'].numpy()
    gt_instance_ids = target['instance_ids'].numpy()
    
    print(f"Ground truth:")
    for j, (label, inst_id, parent_id) in enumerate(zip(gt_labels, gt_instance_ids, gt_parent_ids)):
        if label == 2:  # параллелепипед
            print(f"  Para [{inst_id}] -> Cube [{parent_id}]")

## 11. Сохранение модели

In [ ]:
# Сохранение финальной модели
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'epochs': NUM_EPOCHS,
    'train_losses': train_losses,
    'val_accuracies': val_accuracies,
    'categories': CATEGORIES
}, 'cube_parallelepiped_model.pth')

print("Model saved to cube_parallelepiped_model.pth")

## 12. Inference на новых изображениях

In [ ]:
def inference(model, image_path, device, score_threshold=0.5):
    """Запуск inference на одном изображении."""
    
    # Загрузка изображения
    image = Image.open(image_path).convert("RGB")
    image_tensor = torch.as_tensor(np.array(image), dtype=torch.float32).permute(2, 0, 1) / 255.0
    
    model.eval()
    with torch.no_grad():
        prediction = model([image_tensor.to(device)])[0]
    
    # Фильтрация по score
    keep = prediction['scores'] >= score_threshold
    filtered_pred = {
        'boxes': prediction['boxes'][keep],
        'labels': prediction['labels'][keep],
        'scores': prediction['scores'][keep],
        'masks': prediction['masks'][keep]
    }
    
    # Предсказание связей
    associations = predict_associations(
        filtered_pred['boxes'], 
        filtered_pred['labels'],
        filtered_pred['scores']
    )
    
    # Визуализация
    visualize_predictions(image_tensor, filtered_pred, score_threshold)
    
    print(f"\nDetected objects: {len(filtered_pred['labels'])}")
    print(f"  Cubes: {(filtered_pred['labels'] == 1).sum().item()}")
    print(f"  Parallelepipeds: {(filtered_pred['labels'] == 2).sum().item()}")
    print(f"Associations: {associations}")
    
    return filtered_pred, associations

# Пример использования:
# result, assoc = inference(model, 'path/to/new/image.png', DEVICE)